In [1]:
import os
# !pip install openai
import openai
import matplotlib.pyplot as plt
import numpy as np
import re
import matplotlib.pyplot as plt
openai.api_key = "XXXXXXXXX"
from sklearn.datasets import fetch_openml
import time
from sklearn import tree
import pandas as pd
import random
from collections import Counter
import jsonlines
from datetime import date
import csv
import json
import csv
import random
import openai

### 1. OUM

#### 1.1. Read the Data

In [2]:
def load_data_oum(label='after'):
    final_convs = []
    final_labels = []
    wizards_data = []
    moral_foundations = ["care", "fairness", "liberty", "loyalty", "authority", "sanctity", "none"]
    input_files = {"wizards": "wizards_dialogues.json", "final_argubot": "argubot_final_exp.json",
                   "models_dialogues": "models_dialogues.json"}
    dials_with_scores = {"wizards": {}, "final_argubot": {}, "models_dialogues": {}}

    for key in input_files:
        input_file = input_files[key]
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for d in data:
            is_wiki = False
            for m in d["messages"]:
                if 'model' in m and (m['model'] == 'wikibot' or m['model'] == 'controlbot'):
                    is_wiki = True
                    break
            if is_wiki:
                continue
            yes_no = 'none'
            k = 'Did you vote for (Leave) or against (Remain) Brexit in the 2016 UK referendum?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'against (remain)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'for (leave)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'

            k = 'In the referendum on whether the UK should remain a member of the EU (BREXIT), how did you vote?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'remain (against brexit)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'leave (for brexit)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'
            k = 'Have you had at least one dose of an approved Covid-19 vaccine?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'
            k = 'Are you a vegan?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'

            if yes_no == 'none':
                continue

            if 'Questions' in d['participant_info']:
                for q in d['participant_info']['Questions']:
                    if "final" in input_file:
                        if label == 'oum':
                            continue
                        if d['participant_info']['Questions'][q]['after'] == -1:
                            continue
                    elif d['participant_info']['Questions'][q]['before'] == -1 or d['participant_info']['Questions'][q]['after'] == -1:
                        continue
                    if 'good reasons' in q.lower():
                        if d['topic'] != 'brexit' and 'not' in q.lower() and yes_no == 'no':
                            continue
                        if d['topic'] != 'brexit' and 'not' not in q.lower() and yes_no == 'yes':
                            continue
                        if 'leave' in q.lower() and yes_no == 'yes':
                            continue
                        if 'remain' in q.lower() and yes_no == 'no':
                            continue
                        if d["_id"] not in dials_with_scores[key]:
                            text = ''
                            dials_with_scores[key][d["_id"]] = {"topic": d["topic"], "dataset": key}
                            for message in d['messages']:
                                if message['role'] == 'admin' or 'modified_argument' not in message:
                                    continue

                                text = text + '\n\n' + '<' + message['role'] + '>' + '\n' + message['modified_argument']
                            dials_with_scores[key][d["_id"]]['text'] = text.strip()
                            final_convs.append(text.strip())

                    if 'good reasons' in q.lower():
                        if False and label == 'oum':
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']) - float(d['participant_info']['Questions'][q]['before']))
                        else:
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']))
                        oum = d['participant_info']['Questions'][q]['after'] - d['participant_info']['Questions'][q]['before'] if "final" not in input_file else None
                        dials_with_scores[key][d["_id"]]["good_reasons"] = {"oum": oum, "after": d['participant_info']['Questions'][q]['after']}
                        if 'before' in d['participant_info']['Questions'][q] and d['participant_info']['Questions'][q]['before'] != -1:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = d['participant_info']['Questions'][q]['before']
                        else:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = None

    assert len(final_convs) == len(final_labels)
    return final_convs, final_labels



def get_utterances_oum(text):
    # Splitting the input text into lines
    lines = text.split('\n')
    # Variable to keep the cleaned lines
    cleaned_lines = []
    # Variable to keep track of whether the next line should be added
    add_next_line = False
    for line in lines:
        # If the line is a participant tag, set the flag to add the next line
        if line.strip() == '<participant>':
            add_next_line = True
        elif line.strip() in ['<woz>', '<chatbot>']:
            add_next_line = True
        elif add_next_line:
            # If the flag is set, add this line to the cleaned list and reset the flag
            cleaned_lines.append(line)
            add_next_line = False
    # Join the cleaned lines back into a single string
    cleaned_text = '\n\n'.join(cleaned_lines)
    return cleaned_text

### Bot/Woz's data
conversations, labels = load_data_oum(label='after')
utterances = [get_utterances_oum(c) for c in conversations]


#### 1.2. Run the Experiment

##### 1.2.1. Few-shot

In [ ]:
def gpt4o_fewshot_prompting(conversations, labels, n_shot, seed=123):
    # Prepare the CSV file
    with open(f'./gpt4o_nshot_baseline_results/gpt4o_{n_shot}shot_performance_oum_seed={seed}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

        # Iterate through all conversations
        for i, (conversation, label) in enumerate(zip(conversations, labels)):
            print('\n\nINSTANCE: ', i)
            # Select N random examples (excluding the current one)
            examples = random.sample(list(zip(conversations[:i] + conversations[i+1:], 
                                              labels[:i] + labels[i+1:])), n_shot)
            
            # Construct the prompt
            # prompt = "Given a conversation, predict the score after the conversation. The score corresponds to a post-conversation open-mindedness, obtained by human participants rating their open-mindedness towards the opposing stance on a given controversial topic (veganism, Brexit or vaccination) on a 7-point Likert scale, assessing whether they believe people with opposing views to theirs have good reasons. The score is obtained after the human participant had a discussion with another entity (another human or a chatbot disguised with Wizard of Oz approach). Note, you must only output a single score between 1 to 7 and nothing else.\n\n"
            prompt = """Predict the post-conversation open-mindedness score based on the given conversation. This score reflects a participant's self-reported level of open-mindedness towards opposing views on a controversial topic (veganism, Brexit, or vaccination) after discussing with another entity (human or disguised chatbot).
            
More specifically, the score is based on a 7-point Likert scale, where participants rate whether they believe people with opposing views to theirs have good reasons.

Key points:
1. Score range: 1 to 7
2. Higher scores indicate greater open-mindedness
3. Scores were self-reported after the conversation
4. Topics: veganism, Brexit, or vaccination

You will see example conversation/s with their corresponding scores before predicting the score for a new conversation.

Important: Your response should consist of a single number, with no additional text.

Examples:
            """
            for ex_conv, ex_label in examples:
                prompt += f"CONVERSATION:\n{ex_conv}\n\nSCORE: {ex_label}\n\n\n\n"
            
            # prompt += f"Now, predict the score for this conversation:\n{conversation}\nScore:"
            prompt += f"CONVERSATION:\n{conversation}\n\nSCORE:"
            # print(prompt)

            # Call GPT-4o
            ok = False
            while not ok:
                try:
                    response = openai.ChatCompletion.create(
                        model='gpt-4o',
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=10,
                        temperature=0
                    )
    
                    # Extract the predicted label
                    predicted_label = float(response.choices[0].message.content.strip())
                    ok = True
                except Exception as e:
                    print(f"Error occurred: {e}")
                    predicted_label = "Error"
                    time.sleep(5)
            print('\nPREDICTED SCORE: ', predicted_label)
            # Write to CSV
            csvwriter.writerow([conversation, label, predicted_label])

            # Flush the CSV file to save progress
            csvfile.flush()

            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1} conversations")

for n_shot in [1, 2, 5, 10, 20]:
    gpt4o_fewshot_prompting(conversations, labels, n_shot)

##### 1.2.2. Zero-shot

In [ ]:
for seed in [42, 123, 456]:
    random.seed(seed)
    def gpt4o_zero_prompting(conversations, labels):
        # Prepare the CSV file
        with open(f'./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed={seed}.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

            # Iterate through all conversations
            for i, (conversation, label) in enumerate(zip(conversations, labels)):
                print('\n\nINSTANCE: ', i)

                
                # Construct the prompt
                # prompt = "Given a conversation, predict the score after the conversation. The score corresponds to a post-conversation open-mindedness, obtained by human participants rating their open-mindedness towards the opposing stance on a given controversial topic (veganism, Brexit or vaccination) on a 7-point Likert scale, assessing whether they believe people with opposing views to theirs have good reasons. The score is obtained after the human participant had a discussion with another entity (another human or a chatbot disguised with Wizard of Oz approach). Note, you must only output a single score between 1 to 7 and nothing else.\n\n"
                prompt = """Predict the post-conversation open-mindedness score based on the given conversation. This score reflects a participant's self-reported level of open-mindedness towards opposing views on a controversial topic (veganism, Brexit, or vaccination) after discussing with another entity (human or disguised chatbot).
                
More specifically, the score is based on a 7-point Likert scale, where participants rate whether they believe people with opposing views to theirs have good reasons.

Key points:
1. Score range: 1 to 7
2. Higher scores indicate greater open-mindedness
3. Scores were self-reported after the conversation
4. Topics: veganism, Brexit, or vaccination

Important: Your response should consist of a single number, with no additional text.

                """
                # prompt += f"Now, predict the score for this conversation:\n{conversation}\nScore:"
                prompt += f"CONVERSATION:\n{conversation}\n\nSCORE:"
                # print(prompt)

                # Call GPT-4o
                ok = False
                while not ok:
                    try:
                        response = openai.ChatCompletion.create(
                            model='gpt-4o',
                            messages=[
                                {"role": "user", "content": prompt}
                            ],
                            max_tokens=3,
                            temperature=0
                        )
        
                        # Extract the predicted label
                        predicted_label = float(response.choices[0].message.content.strip())
                        ok = True
                    except Exception as e:
                        print(f"Error occurred: {e}")
                        predicted_label = "Error"
                        time.sleep(5)
                print('\nPREDICTED SCORE: ', predicted_label)
                # Write to CSV
                csvwriter.writerow([conversation, label, predicted_label])

                # Flush the CSV file to save progress
                csvfile.flush()

                # Print progress
                if (i + 1) % 10 == 0:
                    print(f"Processed {i + 1} conversations")

    # Assuming conversations and labels are already loaded
    gpt4o_zero_prompting(conversations, labels)

#### 1.3. Evaluation

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Load the CSV file
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=42.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=123.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=456.csv')
# Calculate Spearman correlation
spearman_corr, _ = stats.spearmanr(df['True Label'], df['Predicted Label'])

# Calculate Mean Absolute Error (MAE)
mae = np.mean(np.abs(df['True Label'] - df['Predicted Label']))

# Print results
print(f"Spearman correlation: {spearman_corr}")
print(f"Mean Absolute Error: {mae}")

In [37]:
def conv2topic(c):
  if 'Brexit' in c:
    return 'Brexit'
  elif 'veganism' in c or 'vegan' in c:
    return 'veganism'
  elif 'vaccination' in c or 'vaccine' in c:
    return 'vaccination'
  else:
    raise 'NO TOPIC WAS CHOSEN'

df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=42.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=123.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_oum_seed=456.csv')

# Convert conversation column to topics
df['Topic'] = df['Conversation'].apply(conv2topic)

# Split the data based on topics
topics = df['Topic'].unique()

# Calculate performance metrics for each topic
for topic in topics:
    topic_df = df[df['Topic'] == topic]
    
    # Calculate Spearman correlation
    spearman_corr, _ = stats.spearmanr(topic_df['True Label'], topic_df['Predicted Label'])
    
    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(topic_df['True Label'] - topic_df['Predicted Label']))
    
    # Print results
    print(f"Topic: {topic}")
    print(f"Spearman correlation: {spearman_corr}")
    print(f"Mean Absolute Error: {mae}")
    print("\n")

Topic: vaccination
Spearman correlation: 0.07339702140983752
Mean Absolute Error: 1.8095238095238095


Topic: Brexit
Spearman correlation: 0.40462835692859317
Mean Absolute Error: 1.6979865771812082


Topic: veganism
Spearman correlation: 0.21971562622189889
Mean Absolute Error: 2.1323529411764706




### 2. Wikitactics

#### 2.1. Load the Data

In [7]:
import json
import pandas as pd
import numpy as np
from collections import Counter

def load_data_wikitac():
    with open('./wikitactics.json') as f:
        data = json.load(f)

    conversations = []
    utterances_cleaned = []
    labels = []
    # Extract conversations/disputes for ESCALATED disputes
    for dispute in data:
        users = list()
        conversation = ''
        utt_cleaned = ''
        for utterance in dispute['utterances']:
            username = utterance['username']
            text = utterance['text']
            conversation += f"<user_id={username}>\n{text}\n" 
            utt_cleaned += text + '\n\n'
        conversations.append(conversation)
        utterances_cleaned.append(utt_cleaned)
        labels.append(dispute['escalation_label'])

    return conversations, utterances_cleaned, labels

conversations, _, labels = load_data_wikitac()

#### 2.2. Run the Expriment

##### 2.2.1. Few-shot

In [ ]:
def gpt4o_fewshot_prompting(conversations, labels, n_shot, seed=123):
    # Prepare the CSV file
    random.seed(seed)
    with open(f'./gpt4o_nshot_baseline_results/gpt4o_{n_shot}shot_performance_wikitactics_seed={seed}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

        # Iterate through all conversations
        for i, (conversation, label) in enumerate(zip(conversations, labels)):
            print('\n\nINSTANCE: ', i)
            # Select 20 random examples (excluding the current one)
            examples = random.sample(list(zip(conversations[:i] + conversations[i+1:], 
                                              labels[:i] + labels[i+1:])), n_shot)
            
            # Construct the prompt
            # prompt = "Given a conversation, predict the score after the conversation. The score corresponds to a post-conversation open-mindedness, obtained by human participants rating their open-mindedness towards the opposing stance on a given controversial topic (veganism, Brexit or vaccination) on a 7-point Likert scale, assessing whether they believe people with opposing views to theirs have good reasons. The score is obtained after the human participant had a discussion with another entity (another human or a chatbot disguised with Wizard of Oz approach). Note, you must only output a single score between 1 to 7 and nothing else.\n\n"
            prompt = """When there is a content accuracy dispute or a violation of Wikipedia’s neutral point of view policy, an editor can create a ‘dispute’ for a potentially problematic article, in which they provide their rationale, vote and discuss them with others. If the editors cannot reach an agreement, they can request mediation from a community volunteer, which is considered an escalation. 

Below you have one dispute sourced from the Wikipedia Talk Page for which you have to predict escalation. We model this as a binary classification task by taking a textual dialogue as input to predict whether the dispute was eventually ‘escalated’ (y=1) or ‘non-escalated’ (y=0).
            
You will see example disputes/s with their corresponding escalation outcome before predicting the escalation for a new dispute.

Important: Your prediction should consist of a single number between 0 and 1, with no additional text.

Examples:
"""
            for ex_conv, ex_label in examples:
                prompt += f"DISPUTE:\n{ex_conv}\n\nESCALATION: {ex_label}\n\n\n\n"
            
            # prompt += f"Now, predict the score for this conversation:\n{conversation}\nScore:"
            prompt += f"DISPUTE:\n{conversation}\n\nESCALATION:"
            # print(prompt)
            # assert 1 == 0
            # Call GPT-4o
            ok = False
            while not ok:
                try:
                    response = openai.ChatCompletion.create(
                        model='gpt-4o',
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=10,
                        temperature=0
                    )
    
                    # Extract the predicted label
                    predicted_label = float(response.choices[0].message.content.strip())
                    ok = True
                except Exception as e:
                    print(f"Error occurred: {e}")
                    predicted_label = "Error"
                    time.sleep(5)
            print('\nPREDICTED SCORE: ', predicted_label)
            # Write to CSV
            csvwriter.writerow([conversation, label, predicted_label])

            # Flush the CSV file to save progress
            csvfile.flush()

            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1} conversations")


for n_shot in [1, 2, 5, 10, 20]: 
    gpt4o_fewshot_prompting(conversations, labels, n_shot)

##### 2.2.2. Zero-shot

In [ ]:
def gpt4o_zeroshot_prompting(conversations, labels, n_shot, seed):
    random.seed(seed)
    # Prepare the CSV file
    with open(f'./gpt4o_nshot_baseline_results/gpt4o_{n_shot}shot_performance_wikitactics_seed={seed}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

        # Iterate through all conversations
        for i, (conversation, label) in enumerate(zip(conversations, labels)):
            print('\n\nINSTANCE: ', i)
            
            # Construct the prompt
            # prompt = "Given a conversation, predict the score after the conversation. The score corresponds to a post-conversation open-mindedness, obtained by human participants rating their open-mindedness towards the opposing stance on a given controversial topic (veganism, Brexit or vaccination) on a 7-point Likert scale, assessing whether they believe people with opposing views to theirs have good reasons. The score is obtained after the human participant had a discussion with another entity (another human or a chatbot disguised with Wizard of Oz approach). Note, you must only output a single score between 1 to 7 and nothing else.\n\n"
            prompt = """When there is a content accuracy dispute or a violation of Wikipedia’s neutral point of view policy, an editor can create a ‘dispute’ for a potentially problematic article, in which they provide their rationale, vote and discuss them with others. If the editors cannot reach an agreement, they can request mediation from a community volunteer, which is considered an escalation. 

Below you have one dispute sourced from the Wikipedia Talk Page for which you have to predict escalation. We model this as a binary classification task by taking a textual dialogue as input to predict whether the dispute was eventually ‘escalated’ (y=1) or ‘non-escalated’ (y=0).
            
Important: Your prediction should consist of a single number between 0 and 1, with no additional text.

"""

            # prompt += f"Now, predict the score for this conversation:\n{conversation}\nScore:"
            prompt += f"DISPUTE:\n{conversation}\n\nESCALATION:"
            # print(prompt)
            # assert 1 == 0
            # Call GPT-4o
            ok = False
            while not ok:
                try:
                    response = openai.ChatCompletion.create(
                        model='gpt-4o',
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=10,
                        temperature=0
                    )
    
                    # Extract the predicted label
                    predicted_label = float(response.choices[0].message.content.strip())
                    ok = True
                except Exception as e:
                    print(f"Error occurred: {e}")
                    predicted_label = "Error"
                    time.sleep(5)
            print('\nPREDICTED SCORE: ', predicted_label)
            # Write to CSV
            csvwriter.writerow([conversation, label, predicted_label])

            # Flush the CSV file to save progress
            csvfile.flush()

            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1} conversations")

# Assuming conversations and labels are already loaded
for seed in [42, 123, 456]:
    gpt4o_zeroshot_prompting(conversations, labels, 0, seed)

#### 2.3. Evaluation

In [23]:
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score

# Read the CSV file

df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_wikitactics_seed=42.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_wikitactics_seed=123.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_wikitactics_seed=456.csv')
# Extract true labels and predicted labels
true_labels = df['True Label']
predicted_labels = df['Predicted Label']

# Calculate AUROC
auroc = roc_auc_score(true_labels, predicted_labels)

# Calculate AUPR
aupr = average_precision_score(true_labels, predicted_labels)

print(f"AUROC: {auroc}")
print(f"AUPR: {aupr}")

AUROC: 0.652442250044084
AUPR: 0.5947221649702495


### 3. AFD

#### 3.1. Read the data

In [12]:
def load_data_afd():
    # Load the data from the JSON file
    with open('afd_1000_randomised_dialogues.json', 'r') as json_file:
        data_dict = json.load(json_file)

    # Extract the conversations, utterances, and labels from the data dictionary
    conversations = data_dict['conversations']
    utterances = data_dict['utterances']
    labels = data_dict['labels']
    labels = [1 if i == 0 else 0 for i in labels]
    return conversations, utterances, labels

conversations, _, labels = load_data_afd()

#### 3.2. Run the Experiment

##### 3.2.1. Few-shot

In [ ]:
random.seed(123) 
def gpt4o_fewshot_prompting(conversations, labels, n_shot):
    # Prepare the CSV file
    with open(f'gpt4o_{n_shot}shot_performance_afd_seed=123.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

        # Iterate through all conversations
        for i, (conversation, label) in enumerate(zip(conversations, labels)):
            print('\n\nINSTANCE: ', i)
            # Select N random examples (excluding the current one)
            examples = random.sample(list(zip(conversations[:i] + conversations[i+1:], 
                                              labels[:i] + labels[i+1:])), n_shot)
            
            # Construct the prompt
            prompt = """I will show you a debate where Wikipedia editors discuss whether to delete certain Wikipedia articles. 
            
The debate (which lasts at least seven days) is initiated by a Wikipedia editor, who provides reasoning for nominating an article for deletion and then discusses it with other editors. Administrators then aggregate the discussion to decide the outcome. 

For our analysis, we model a binary classification task by including articles labelled as ‘Delete’ (y=0) and ‘Keep’ (y=1).
            
You will see example debate/s with their corresponding outcome before predicting whether a new given dispute will be kept or deleted.

Important: Your prediction should consist of a single number, with no additional text.

Examples:
"""
            for ex_conv, ex_label in examples:
                prompt += f"DEBATE:\n{ex_conv}\n\nOUTCOME: {ex_label}\n\n\n\n"
            
            prompt += f"DEBATE:\n{conversation}\n\nOUTCOME:"
            ok = False
            while not ok:
                try:
                    response = openai.ChatCompletion.create(
                        model='gpt-4o',
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=10,
                        temperature=0
                    )
    
                    # Extract the predicted label
                    predicted_label = float(response.choices[0].message.content.strip())
                    ok = True
                except Exception as e:
                    print(f"Error occurred: {e}")
                    predicted_label = "Error"
                    time.sleep(5)
            print('\nPREDICTED SCORE: ', predicted_label)
            # Write to CSV
            csvwriter.writerow([conversation, label, predicted_label])

            # Flush the CSV file to save progress
            csvfile.flush()

            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1} conversations")

for n_shot in [1, 2, 5, 10, 20]:
    gpt4o_fewshot_prompting(conversations, labels, n_shot)

##### 3.2.2. Zero-shot

In [ ]:
def gpt4o_0shot_prompting(conversations, labels, nshot, seed):
    # Prepare the CSV file
    random.seed(seed)
    with open(f'./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_afd_seed={seed}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Conversation', 'True Label', 'Predicted Label'])

        # Iterate through all conversations
        for i, (conversation, label) in enumerate(zip(conversations, labels)):
            print('\n\nINSTANCE: ', i)


            prompt = """Below is a discussion between different individuals who discuss whether to delete a given Wikipedia article. 
            
The debate is initiated by an individual, who provides reasoning for nominating an article for deletion and then discusses it with other individuals. Administrators will in the end aggregate the discussion to decide whether to keep or delete the article. 

This is modelled as a binary classification task, taking an input discussion to predict if the outcome will be ‘Keep’ (y=1) or ‘Delete’ (y=0).
            
Important: Your prediction should consist of a single number, with no additional text.

"""

            prompt += f"DEBATE:\n{conversation}\n\nOUTCOME:"

            ok = False
            while not ok:
                try:
                    response = openai.ChatCompletion.create(
                        model='gpt-4o',
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=3,
                        temperature=0
                    )
    
                    # Extract the predicted label
                    predicted_label = float(response.choices[0].message.content.strip())
                    ok = True
                except Exception as e:
                    print(f"Error occurred: {e}")
                    predicted_label = "Error"
                    time.sleep(5)
            print('\nPREDICTED SCORE: ', predicted_label)
            # Write to CSV
            csvwriter.writerow([conversation, label, predicted_label])

            # Flush the CSV file to save progress
            csvfile.flush()

            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1} conversations")

# Assuming conversations and labels are already loaded
for seed in [42, 123, 456]:
    gpt4o_0shot_prompting(conversations, labels, 0, seed)

#### 3.3. Evaluation

In [33]:
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score

# Read the CSV file
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_afd_seed=42.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_0shot_performance_afd_seed=123.csv')
df = pd.read_csv('./gpt4o_nshot_baseline_results/gpt4o_5shot_performance_afd_seed=456.csv')
# Extract true labels and predicted labels
true_labels = df['True Label']
predicted_labels = df['Predicted Label']

# Calculate AUROC
auroc = roc_auc_score(true_labels, predicted_labels)

# Calculate AUPR
aupr = average_precision_score(true_labels, predicted_labels)

print(f"AUROC: {auroc}")
print(f"AUPR: {aupr}")

AUROC: 0.9169574129396628
AUPR: 0.8447070920622718


In [39]:
# Provided values for calculation
auroc_values = [0.9057055321747094, 0.9119038792821473, 0.9080474279857824]
aupr_values = [0.85086818389468, 0.8602636445462729, 0.8541563197681251]

# Calculate mean and standard deviation for AUROC
auroc_mean = np.mean(auroc_values)
auroc_sd = np.std(auroc_values, ddof=1)

# Calculate mean and standard deviation for AUPR
aupr_mean = np.mean(aupr_values)
aupr_sd = np.std(aupr_values, ddof=1)

# Round the results to three decimal places
auroc_mean_rounded = round(auroc_mean, 3)
auroc_sd_rounded = round(auroc_sd, 3)
aupr_mean_rounded = round(aupr_mean, 3)
aupr_sd_rounded = round(aupr_sd, 3)

auroc_mean_rounded, auroc_sd_rounded, aupr_mean_rounded, aupr_sd_rounded


(0.909, 0.003, 0.855, 0.005)